# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [84]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import gmaps
import os
import json

# Import API key
from api_keys import geoapify_key

In [85]:
# Load the CSV file created in Part 1 into a Pandas DataFrame

city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


In [86]:
city_data_df.count()

City_ID       577
City          577
Lat           577
Lng           577
Max Temp      577
Humidity      577
Cloudiness    577
Wind Speed    577
Country       574
Date          577
dtype: int64

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [87]:
max_hum=city_data_df["Humidity"].max()
max_hum

100

In [88]:
%%capture --no-display

# Configure the map plot
gmaps.configure(api_key=geoapify_key)

#get locations and humidity
city_locations = city_data_df[["Lat", "Lng"]].astype(float)
humidity = city_data_df['Humidity'].astype(float)

#format layout of map
layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'}

#add heatmap
fig = gmaps.figure(map_type='HYBRID', layout=layout)

heat_layer = gmaps.heatmap_layer(city_locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1000px'))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [89]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = city_data_df

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# A max temperature lower than 80 degrees but higher than 70.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temp"] < 80) & (filtered_weather_df["Max Temp"] > 70)]

# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


In [90]:
#filtered_weather_df = city_data_df.loc[(city_data_df["Max Temp"] >= 70) & (city_data_df["Max Temp"] <= 80) & 
                                  #(city_data_df['Wind Speed'] <= 20) & (city_data_df['Humidity'] <= 30)]
#filtered_weather_df

### Step 3: Create a new DataFrame called `hotel_df`.

In [91]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_weather_df
#hotel_df.reset_index(drop=True, inplace=True)
hotel_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": geoapify_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

# Display sample data
print(response)


{'error_message': 'The provided API key is invalid.', 'html_attributions': [], 'results': [], 'status': 'REQUEST_DENIED'}


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [92]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "radius": radius,
    "types": "lodging",
    "key": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["location"] = f"{lat}, {lng}"
    
    # Set base URL
    #base_url = "https://api.geoapify.com/v2/places"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make and API request using the params dictionaty
    name_address = params["location"]
    
    # Convert the API response to JSON format
    name_address = requests.get(base_url).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [93]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE